In [ ]:
import pandas as pd
import os
import re
import nltk
import numpy as np
import sklearn

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import files
_ = files.upload()

Saving 040920 Description 2d animator.csv to 040920 Description 2d animator.csv
Saving 040920 Description Motion Designer.csv to 040920 Description Motion Designer.csv
Saving 080920 Description 2d animator.csv to 080920 Description 2d animator.csv
Saving 080920 Description Motion Designer.csv to 080920 Description Motion Designer.csv
Saving 200820 Description 2d animator.csv to 200820 Description 2d animator.csv
Saving 200820 Description Motion Designer.csv to 200820 Description Motion Designer.csv
Saving 200820 description test.csv to 200820 description test.csv
Saving 240820 Description 2d animator.csv to 240820 Description 2d animator.csv
Saving 240820 Description Motion Designer.csv to 240820 Description Motion Designer.csv
Saving 260820 Description 2d animator.csv to 260820 Description 2d animator.csv
Saving 260820 Description Motion Designer.csv to 260820 Description Motion Designer.csv
Saving 290820 Description 2d animator.csv to 290820 Description 2d animator.csv
Saving 290820 

In [ ]:
from collections import OrderedDict

MIN_HEADER_LEN = 0
MAX_HEADER_LEN = 5000
# trying to not check header length
MIN_SECTIONS = 2
MAX_SECTIONS = 10

def strip_for_all(list_of_str : list, func=str.strip, sensfull_length=4):
    return [func(elem) for elem in list_of_str if len(elem) >= sensfull_length]

def return_first_items(input_dict : OrderedDict):
    try:
        first_out_key = list(input_dict.keys())[0]
    except IndexError:
        return (OrderedDict())
    try:
        first_inn_key = list(input_dict[first_out_key].keys())[0]
    except IndexError:
        return (OrderedDict())
    return (input_dict[first_out_key][first_inn_key])

def segmentation_by_terminals(full_text : str, keyword_to_paragraph : {}, terminals=["**"], small_terminals=["*", r"\-", "\n"]):
    # trying cut full text in segments [T,h,T,h,T], where h - headear, T - text
    # then evaluate quality of cutting

    segmentations_variants = OrderedDict()

    #TODO: Check results by different terminals
    for terminal in terminals:
        segmentations_variants[terminal] = OrderedDict()
        segments = full_text.split(terminal)
        segments = strip_for_all(segments)

        for sub_terminal in small_terminals:
            segmentations_variants[terminal][sub_terminal] = OrderedDict()
            max_sections = 0

            for i in range(len(segments)):
                sub_segments = segments[i].split(sub_terminal)
                sub_segments = strip_for_all(sub_segments)
                if len(sub_segments) >= MIN_SECTIONS:
                    if i > 0:
                        if MIN_HEADER_LEN <= len(segments[i-1]) <= MAX_HEADER_LEN:
                            segmentations_variants[terminal][sub_terminal][segments[i-1]] = sub_segments
                            if len(sub_segments) > max_sections:
                                max_sections = len(sub_segments)
                    elif i == 0:
                        segmentations_variants[terminal][sub_terminal]["headless"] = sub_segments

            if not segmentations_variants[terminal][sub_terminal]:
                del segmentations_variants[terminal][sub_terminal]
            if max_sections >= MAX_SECTIONS:
                del segmentations_variants[terminal][sub_terminal]

        if not segmentations_variants[terminal]:
            del segmentations_variants[terminal]

    return (segmentations_variants)

def is_nan_or_empty(a : str):
    if (a != a):
        return (True)
    else:
        return (not bool(a))

In [ ]:
df = pd.DataFrame()

for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith("csv"):
            print (filename)
            temp_df = pd.read_csv(filename)
            df = pd.concat([df, temp_df])
    break

df.head()

080920 Description 2d animator.csv
290820 Description 2d animator.csv
240820 Description 2d animator.csv
260820 Description Motion Designer.csv
240820 Description Motion Designer.csv
260820 Description 2d animator.csv
290820 Description Motion Designer.csv
080920 Description Motion Designer.csv
040920 Description Motion Designer.csv
200820 Description 2d animator.csv
200820 Description Motion Designer.csv
200820 description test.csv
040920 Description 2d animator.csv


,Unnamed: 0,description,snippet.requirement,snippet.responsibility,Unnamed: 4,Unnamed: 5
0,0,Всем привет! Мы компания Adme и сейчас мы в по...,Отличное владение After Effects (также рассмат...,NaN,NaN,NaN
1,0,\n\nWhaleapp — ведущая компания в мире создани...,Имеешь опыт работы со скелетной анимацией от 1...,Создание рига и анимаций в Spine 2D. Создание ...,NaN,NaN
2,0,"\n\nMotionads — креативная студия , специализи...","Мы ждем, что ты уверенно работаешь в After Eff...",NaN,NaN,NaN
3,0,\n\n **Чем предстоит заниматься:**\n\n * Созд...,Опыт работы на позиции аниматора от полугода. ...,Создание анимаций для мобильных игр в Spine: м...,NaN,NaN
4,0,\n\n **Задачи:**\n\n * Создание анимаций для ...,Опыт работы на позиции аниматора от полугода. ...,Создание анимаций для мобильных игр в Spine: м...,NaN,NaN


In [ ]:
class BagOfWords_Vectorizer():

    def __init__(self, full_text : str, languages=["russian","english"], DEBUG=True):
        self.DEBUG = DEBUG
        self.vacabulary = re.sub(r"[^a-zA-ZА-Яа-яЁё]", " ", full_text.lower())
        self.stopwords = nltk.corpus.stopwords.words(languages)
        self.vacabulary = sorted(set(nltk.tokenize.word_tokenize(self.vacabulary)) - set(self.stopwords))

    def str_to_vec(self, target_str : str):
        get_words = re.sub(r"[^a-zA-ZА-Яа-яЁё]", " ", target_str.lower())
        get_words = set(nltk.tokenize.word_tokenize(get_words)) - set(self.stopwords)
        vector = np.zeros(len(self.vacabulary))
        indexes = []
        for word in get_words:
            try:
                indexes.append(self.vacabulary.index(word))
            except ValueError:
                if self.DEBUG:
                    message = 'there is no such word as "{}" in dictinary. There is some mistake here'.format(word)
                    raise ValueError(message)
                else:
                    print ('there is no such word as "{}" in dictinary. There is some mistake here'.format(word))
        vector[indexes] = 1
        return (vector)

    def vec_to_str(self, vector : np.array):
        words = []
        for i in range(len(vector)):
            if vector[i]:
                words.append(self.vacabulary[i])
        return (words)

    def str_to_str(self, target_str : str):
        get_words = re.sub(r"[^a-zA-ZА-Яа-яЁё]", " ", target_str.lower())
        get_words = set(nltk.tokenize.word_tokenize(get_words)) - set(self.stopwords)
        return (get_words)

In [ ]:
def get_marks_for_headers(df : pd.DataFrame, vectorizer : BagOfWords_Vectorizer, DEBUG=False, my_debug=[]):
    X = []
    Y = []

    for i in range(len(df)):
        segments = return_first_items(segmentation_by_terminals(df["description"].iloc[i], keyword_to_paragraph={}))
        tails_vec = []
        heads_vec = []
        indexes = []
        for key in segments.keys():
            tails_vec.append(vectorizer.str_to_vec(" ".join(segments[key])))
            heads_vec.append(vectorizer.str_to_vec(key))
        if tails_vec:
            tails_vec = np.array(tails_vec)
            heads_vec = np.array(heads_vec)
            indexes = list(range(tails_vec.shape[0]))
            #print (indexes)
            if not is_nan_or_empty(df["snippet.requirement"].iloc[i]):
                vec_req = vectorizer.str_to_vec(df["snippet.requirement"].iloc[i])
                cosine = ((tails_vec * vec_req).sum(axis=1)/vec_req.sum())
                index = cosine.argmax()
                X.append(heads_vec[index])
                Y.append(np.array([1,0,0,0]))  # "requirment" class, "responsibility" class, "other" class, "none above" class
                indexes.remove(index)
            if not is_nan_or_empty(df["snippet.responsibility"].iloc[i]):
                vec_res = vectorizer.str_to_vec(df["snippet.responsibility"].iloc[i])
                cosine = ((tails_vec * vec_res).sum(axis=1)/vec_res.sum())
                index = cosine.argmax()
                try:
                    indexes.remove(index)
                    Y.append(np.array([0,1,0,0]))
                    X.append(heads_vec[index])
                except ValueError:
                    if DEBUG:
                        Y.append(np.array([1,1,0,0]))
                        my_debug.append(i)
                        print (my_debug)
                        print ("clashing indexes. Requirements and responsibility vector have same closest vector")
                    else:
                        # just throwing away problem records
                        # there can be paragraphs with one item
                        # special symbols across paragraph
                        # or both snippet in one paragraph
                        del X[-1]
                        del Y[-1]
            for index in indexes:
                X.append(heads_vec[index])
                Y.append(np.array([0,0,1,0]))
    X = np.row_stack(X)
    Y = np.row_stack(Y)
    return (X, Y)


full_text = str()
for i in range(len(df["description"])):
    #TODO: check for none?
    full_text += df["description"].iloc[i]
full_text += " headless" #add my header to vocabulary

vectorizer = BagOfWords_Vectorizer(full_text=full_text, DEBUG=False)
# print ("full text length is",len(full_text))
del (full_text)

X, Y = get_marks_for_headers(df, vectorizer, DEBUG=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(random_state=42, max_depth=10)
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

In [ ]:
(Y_pred == Y_test).sum() / (Y_pred == Y_test).size

0.8992172211350293

In [ ]:
feature_names = vectorizer.vec_to_str(np.ones(len(vectorizer.vacabulary)))
text_representation = tree.export_text(clf, feature_names=feature_names)
print(text_representation)

|--- требования <= 0.50
|   |--- обязанности <= 0.50
|   |   |--- задачи <= 0.50
|   |   |   |--- предстоит <= 0.50
|   |   |   |   |--- experience <= 0.50
|   |   |   |   |   |--- заниматься <= 0.50
|   |   |   |   |   |   |--- ожидаем <= 0.50
|   |   |   |   |   |   |   |--- предлагаем <= 0.50
|   |   |   |   |   |   |   |   |--- условия <= 0.50
|   |   |   |   |   |   |   |   |   |--- fired <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |   |   |   |   |--- fired >  0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- условия >  0.50
|   |   |   |   |   |   |   |   |   |--- работы <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |   |--- работы >  0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- предлагаем >  0.50
|   |   |   |   |   |   |   |   |--- нашего <= 0.50
|   |   |   |   |   |   |   |   |   |--- атмосферы <= 0